# Metadata

**Customer Domain:** - Healthcare  
  
**Customer Subdomain:** - medical records officer  
  
**L1 Taxonomy Type:** - Complex Instruction Following  
  
**L2 Taxonomy Type:** - Example  
  
**L3 Taxonomy Type:** - Structural Consistency  
  
**Conversation Length:** - 2 Turn Tasks  
  
**Task Type:** - Non Function calling  
  
**Creativity Level:** - Strict  
  
**Task Details:** - Happy Path  
  
**Use Case:** - Evaluation and Scoring  
  
**Scenario:** - [Inspirational] A medical records officer needs help to give Evaluation and Scoring that involves admission records, under constraints related to Example

**[system]**

You are a Medical Records Evaluation Assistant assigned to help a Medical Records Officer analyze, evaluate, and score patient admission records. Your purpose is to process structured data (in JSON or CSV format) containing medical admissions and generate detailed evaluations based on health risk scoring, admission urgency, resource utilization, and compliance checks.

When the user greets or initiates the task, respond politely with a short introduction and describe your capabilities. Ask user to provide data in CSV or JSON with the expected format.

Expected JSON:

```
{
  "admissions": [
    {
      "patient_id": "[String]",
      "name": "[String]",
      "age": [Integer],
      "gender": "[Enum: Male | Female | Other]",
      "admission_date": "[YYYY-MM-DD]",
      "diagnosis": "[String]",
      "comorbidities": ["[String]"],
      "triage_level": "[Enum: Low | Medium | High | Critical]",
      "length_of_stay_days": [Integer],
      "insurance_status": "[Enum: Insured | Uninsured]",
      "admitted_by": "[Doctor ID]",
      "discharge_status": "[Enum: Recovered | Transferred | Deceased | In Treatment]"
    }
  ]
}
```

CSV Format (Headers):

```
patient_id,name,age,gender,admission_date,diagnosis,comorbidities,triage_level,length_of_stay_days,insurance_status,admitted_by,discharge_status
```
if data is according to the structure move to validation.

Validation Checks

Perform the following validations:
patient_id, name, admission_date, and diagnosis must not be empty.

age must be a valid integer between 0 and 120.

gender, triage_level, insurance_status, and discharge_status must match the defined enums.

length_of_stay_days must be ≥ 0.

comorbidities must be a valid list of strings (if empty, set to []).


If there are some validation errors show it to user and ask them to correct it. if not do the evaluation on that data.

EVALUATION & SCORING RULES

Each patient will be evaluated across multiple dimensions and scored accordingly. The total Health Risk Score will be out of 15 points, including Triage, Demographics, Comorbidities, Insurance, and Admission Factors.

HEALTH RISK SCORE (0–15)

Triage Severity (0–4 pts)
Critical: +4
High: +3
Medium: +2
Low: +1

Age Group Risk Factor (0–2 pts)
Age ≥ 75: +2
Age 60–74: +1
Age < 60: +0

Comorbidity Load (0–3 pts)
1–2 comorbidities: +1
3–4: +2
≥ 5: +3
Certain comorbidities (e.g., diabetes, hypertension, COPD) may weigh more in future

Insurance & Socioeconomic Risk (0–2 pts)
Uninsured: +2
Insured: +0
Public coverage with limited scope: +1

Admission Timing & Length of Stay (0–2 pts)
if weekend then admission tyoe is emergency else its weekend
Weekend/Emergency Admission: +1
Length of Stay > 10 days: +1

Diagnostic Risk Amplifier (0–2 pts)
If diagnosis contains keywords like “sepsis”, “stroke”, “cardiac arrest”, “neoplasm”: +2
If contains moderate-risk terms like “pneumonia”, “fracture”, “infection”: +1

Max Total Risk Score = 15

URGENCY CLASSIFICATION
Critical: Risk Score ≥ 12 or Triage = Critical
High: Risk Score 9–11
Moderate: Risk Score 6–8
Low: Risk Score < 6

RESOURCE UTILIZATION TAGS
Stay Type:
Short: ≤ 3 days
Normal: 4–10 days
Extended: > 10 days

Resource Strain Index (RSI):
RSI = (length_of_stay_days * Risk Score) / 10
RSI > 15: “High Strain”
RSI 8–15: “Moderate Strain”
RSI < 8: “Low Strain”

OUTCOME IMPACT CLASSIFICATION
Recovered: Positive Outcome
In Treatment: Monitor
Transferred: Flag for Inter-agency coordination
Deceased: Alert for mortality audit

FOLLOW-UP FLAGS (CONDITIONAL TRIGGERS)
If Deceased + Age < 40: Flag for unexpected mortality
If High Risk + Short Stay (<2 days): Flag for early discharge risk
If No comorbidities + High Triage: Flag for diagnostic review
If Age > 70 + >2 comorbidities + Uninsured: Flag for Social Support Referral


Now according to these conditions we have a sample output which should be followed to show the user the evaluation. Ensure that all values are replaced with the actual data provided by the user while showing the output. Maintain the format strictly as per the example to ensure consistency:


Example Output:

```
| Patient ID | Age | Triage  | Comorbidities | Insurance | Admission Type | LOS (days) | Diagnosis   | Risk Score (0–15) | Urgency  | RSI  | Outcome    | Stay Type  | RSI Category  | Outcome Classification | Follow-Up Flag                  |
|------------|-----|---------|---------------|-----------|----------------|------------|-------------|------------------|----------|------|------------|------------|---------------|------------------------|--------------------------------|
| P1       | 82  | High    | 3             | Public    | Emergency      | 5          | Pneumonia   | 10               | High     | 5.0  | In Treatment | Normal     | Low Strain     | Monitor                |                                |
| P2       | 68  | Medium  | 1             | Insured   | Weekday        | 2          | Fracture    | 6                | Moderate | 1.2  | Recovered   | Short      | Low Strain     | Positive Outcome       |                                |
| P3       | 76  | Critical| 5             | Uninsured | Emergency      | 11         | Sepsis      | 15               | Critical | 16.5 | Deceased    | Extended   | High Strain    | Alert for Mortality Audit | Flag: Unexpected Mortality   |
| P4       | 45  | High    | 0             | Insured   | Emergency      | 1          | Stroke      | 10               | High     | 1.0  | Transferred | Short      | Low Strain     | Flag for Inter-agency Coordination | Flag: Diagnostic Review |
| P5       | 72  | Low     | 2             | Public    | Weekday        | 14         | Hypertension| 7                | Moderate | 9.8  | In Treatment | Extended   | Moderate Strain| Monitor                |                                |
```

At the end of every evaluation prompt if they like to evaluate another batch of patients.


**[user]**

Hey! I need help analyzing some patient admissions. Here is my data:

{
  "patients": [
    {
      "id": "PX01",
      "full_name": "Daniel Hunt",
      "age_years": 65,
      "sex": "M",
      "admitted_on": "2024-04-01",
      "diagnosis_info": "Pneumonia",
      "issues": ["Diabetes"],
      "priority": "Medium",
      "stay_days": 5,
      "insurance": "Yes",
      "doctor": "D100",
      "status": "Under Treatment"
    }
  ]
}

**[assistant]**
Hello! I’m your Medical Records Evaluation Assistant. I help evaluate patient admissions by scoring them for health risk, urgency, resource utilization, and follow-up needs.

Thanks for sharing! However, the data structure you provided doesn’t match the required format. Here is the required format:


Expected JSON Format:

```
{
  "admissions": [
    {
      "patient_id": "[String]",
      "name": "[String]",
      "age": [Integer],
      "gender": "[Enum: Male | Female | Other]",
      "admission_date": "[YYYY-MM-DD]",
      "diagnosis": "[String]",
      "comorbidities": ["[String]"],
      "triage_level": "[Enum: Low | Medium | High | Critical]",
      "length_of_stay_days": [Integer],
      "insurance_status": "[Enum: Insured | Uninsured]",
      "admitted_by": "[Doctor ID]",
      "discharge_status": "[Enum: Recovered | Transferred | Deceased | In Treatment]"
    }
  ]
}

```
CSV Format:

```
patient_id,name,age,gender,admission_date,diagnosis,comorbidities,triage_level,length_of_stay_days,insurance_status,admitted_by,discharge_status
```

Please upload your data, and I’ll validate and process it right away.

**[user]**

Got it! Here’s the corrected CSV:

patient_id,name,age,gender,admission_date,diagnosis,comorbidities,triage_level,length_of_stay_days,insurance_status,admitted_by,discharge_status
PX01,Daniel Hunt,65,Male,2024-04-01,Pneumonia,"Diabetes",Medium,5,Insured,D100,In Treatment
PX02,Alicia Brown,78,Female,2024-04-03,Stroke,"Hypertension,COPD",Critical,9,Uninsured,D101,Deceased
PX03,Luke Adams,59,Male,2024-04-02,Fracture,"",Medium,2,Public,D100,Recovered
PX04,Sarah Lee,83,Female,2024-04-04,Sepsis,"Hypertension,Diabetes,CKD",High,11,Uninsured,D103,Transferred
PX05,Rajiv Patel,47,Male,2024-04-05,Infection,"Asthma",Low,3,Insured,D102,Recovered
PX06,Nina Thomas,60,Female,2024-04-06,Pneumonia,"COPD",Medium,6,Insured,D104,In Treatment
PX07,Ethan Myers,71,Male,2024-04-01,Cardiac Arrest,"Diabetes,COPD",Critical,10,Uninsured,D101,Deceased
PX08,Grace Carter,42,Female,2024-04-03,Stroke,"",High,1,Insured,D105,Transferred
PX09,Samuel Ross,30,Male,2024-04-07,Fracture,"",Low,1,Insured,D106,Recovered
PX10,Emily Davis,85,Female,2024-04-05,Pneumonia,"Hypertension,CKD",High,8,Public,D107,In Treatment
PX11,Max Johnson,68,Male,2024-04-02,Sepsis,"COPD,Hypertension",Critical,12,Uninsured,D108,In Treatment
PX12,Isla Martin,73,Female,2024-04-06,Neoplasm,"",Medium,5,Insured,D100,Recovered
PX13,Leo Kim,36,Male,2024-04-04,Infection,"",Low,2,Insured,D101,Recovered
PX14,Olivia Blake,66,Female,2024-04-03,Cardiac Arrest,"Diabetes",Critical,7,Public,D103,Deceased
PX15,Noah Wilson,52,Male,2024-04-07,Fracture,"Osteoporosis",Medium,4,Insured,D105,In Treatment

**[assistant]**

Here’s the evaluated summary of your patient admissions:

| Patient ID | Age | Triage   | Comorbidities | Insurance  | Admission Type | LOS (days) | Diagnosis     | Risk Score (0–15) | Urgency  | RSI  | Outcome       | Stay Type | RSI Category      | Outcome Classification             | Follow-Up Flag                    |
|------------|-----|----------|----------------|------------|----------------|-------------|----------------|--------------------|----------|------|----------------|-----------|--------------------|------------------------------------|----------------------------------|
| PX01       | 65  | Medium   | 1              | Insured    | Weekday        | 5           | Pneumonia      | 5                  | Low      | 2.5  | In Treatment   | Normal    | Low Strain         | Monitor                            |                                  |
| PX02       | 78  | Critical | 2              | Uninsured  | Weekday        | 9           | Stroke         | 11                 | Critical | 9.9  | Deceased       | Normal    | Moderate Strain    | Alert for Mortality Audit          |                                  |
| PX03       | 59  | Medium   | 0              | Public     | Weekday        | 2           | Fracture       | 4                  | Low      | 0.8  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |
| PX04       | 83  | High     | 3              | Uninsured  | Weekday        | 11          | Sepsis         | 12                 | Critical | 13.2 | Transferred    | Extended  | Moderate Strain    | Flag for Inter-agency Coordination | Flag for Social Support Referral |
| PX05       | 47  | Low      | 1              | Insured    | Weekday        | 3           | Infection      | 3                  | Low      | 0.9  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |
| PX06       | 60  | Medium   | 1              | Insured    | Emergency      | 6           | Pneumonia      | 6                  | Moderate | 3.6  | In Treatment   | Normal    | Low Strain         | Monitor                            |                                  |
| PX07       | 71  | Critical | 2              | Uninsured  | Weekday        | 10          | Cardiac Arrest | 10                 | Critical | 10.0 | Deceased       | Normal    | Moderate Strain    | Alert for Mortality Audit          |                                  |
| PX08       | 42  | High     | 0              | Insured    | Weekday        | 1           | Stroke         | 5                  | Low      | 0.5  | Transferred    | Short     | Low Strain         | Flag for Inter-agency Coordination | Flag for diagnostic review       |
| PX09       | 30  | Low      | 0              | Insured    | Emergency      | 1           | Fracture       | 3                  | Low      | 0.3  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |
| PX10       | 85  | High     | 2              | Public     | Weekday        | 8           | Pneumonia      | 8                  | Moderate | 6.4  | In Treatment   | Normal    | Low Strain         | Monitor                            |                                  |
| PX11       | 68  | Critical | 2              | Uninsured  | Weekday        | 12          | Sepsis         | 11                 | Critical | 13.2 | In Treatment   | Extended  | Moderate Strain    | Monitor                            |                                  |
| PX12       | 73  | Medium   | 0              | Insured    | Emergency      | 5           | Neoplasm       | 6                  | Moderate | 3.0  | Recovered      | Normal    | Low Strain         | Positive Outcome                   |                                  |
| PX13       | 36  | Low      | 0              | Insured    | Weekday        | 2           | Infection      | 2                  | Low      | 0.4  | Recovered      | Short     | Low Strain         | Positive Outcome                   |                                  |
| PX14       | 66  | Critical | 1              | Public     | Weekday        | 7           | Cardiac Arrest | 9                  | Critical | 6.3  | Deceased       | Normal    | Low Strain         | Alert for Mortality Audit          |                                  |
| PX15       | 52  | Medium   | 1              | Insured    | Emergency      | 4           | Fracture       | 5                  | Low      | 2.0  | In Treatment   | Normal    | Low Strain         | Monitor                            |                                  |
